<a href="https://colab.research.google.com/github/gowun/BladderCancer_AMC/blob/master/Notebooks/all_process.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import auth
from google.colab import drive

import pandas as pd
import numpy as np

pd.set_option('display.max_colwidth', -1)

auth.authenticate_user()
drive.mount('/content/gdrive')
home_path = '/content/gdrive/My Drive/BladderCancer_AMC/'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  import sys


Mounted at /content/gdrive


In [3]:
!pip install lifelines
!pip install shap
!git clone https://github.com/gowun/BladderCancer_AMC.git

     |████████████████████████████████| 358kB 5.1MB/s 
     |████████████████████████████████| 61kB 5.9MB/s 
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-cp37-none-any.whl size=4050 sha256=027631129a678e4de0eedcb887ea1d3b625211be3a6d356df87119cd202f7816
  Stored in directory: /root/.cache/pip/wheels/dc/68/dc/91321c55fba449755524481854f5be70d41912b8f886f908bb
Successfully built autograd-gamma
     |████████████████████████████████| 358kB 5.0MB/s 
  Created wheel for shap: filename=shap-0.39.0-cp37-cp37m-linux_x86_64.whl size=491618 sha256=75272b3baad5a6b2ae401bfc2c6dd8c115d3e71cd159a1e939ebb1ab9ec7cf52
  Stored in directory: /root/.cache/pip/wheels/15/27/f5/a8ab9da52fd159aae6477b5ede6eaaec69fd130fa0fa59f283
Successfully built shap
Cloning into 'BladderCancer_AMC'...
remote: Enumerating objects: 130, done.
remote: Counting objects: 100% (130/130), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 130 (delta 44), reused 77 (delta 22), pack-reused 

In [4]:
from BladderCancer_AMC.ModelingTools import utils as ut
from BladderCancer_AMC.ModelingTools import clustering as cl
from BladderCancer_AMC.ModelingTools import tree_modeling as tm
from BladderCancer_AMC.ModelingTools import linear_modeling as lm
from BladderCancer_AMC.ModelingTools import figure as fe
from BladderCancer_AMC.ModelingTools import preprocessing as pp

In [5]:
before_md = ut.load_data(home_path + 'meta_datasets.pkl', 'pickle')
before_amc = ut.load_data(home_path + 'amc.pkl', 'pickle')
after_amc = ut.load_data(home_path + 'QuantSeq_R9-21.csv', 'csv')
after_amc = after_amc.set_index('Gene symbol')
after_amc = after_amc.T

In [6]:
after_amc.head()

Gene symbol,DDX11L1,WASH7P,FAM138A,FAM138F,OR4F5,LOC729737,LOC100133331,LOC100132287,LOC100132062,OR4F3,OR4F29,OR4F16,LOC101928626,LOC100288069,FAM87B,LINC00115,LINC01128,FAM41C,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,PERM1,HES4,ISG15,AGRN,RNF223,C1orf159,LINC01342,TTLL10,TNFRSF18,TNFRSF4,SDF4,B3GALT6,FAM132A,UBE2J2,SCNN1D,ACAP3,PUSL1,...,RBMY2EP,RBMY1B,RBMY1A1,RBMY1D,RBMY1E,TTTY13,PRY2,PRY,LOC101929148,TTTY6,TTTY6B,RBMY1F,RBMY1J,TTTY5,RBMY2FP,LOC100652931,TTTY17B,TTTY17C,TTTY17A,TTTY4C,TTTY4B,TTTY4,BPY2C,BPY2B,BPY2,DAZ4,DAZ1,DAZ3,DAZ2,TTTY3B,TTTY3,CDY1B,CDY1,CSPG4P1Y,GOLGA2P3Y,GOLGA2P2Y,SPRY3,VAMP7,IL9R,DDX11L16
R_9,17,17,0,0,0,548,217,0,0,0,0,0,0,0,0,0,0,0,0,450,450,68,0,57,2064,114,3489,0,14,0,0,125,0,2316,170,0,1506,1,455,296,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,107,0,16
R_10,117,123,4,0,0,148,81,0,0,0,0,0,0,4,0,0,30,0,0,177,177,90,204,1,402,154,2457,0,77,0,26,0,0,707,196,0,1230,0,207,356,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,24,0,20
R_11,23,50,0,0,0,46,79,0,0,0,0,0,0,1,0,28,6,0,0,167,167,36,67,123,650,550,1857,0,181,0,0,25,0,1342,200,13,759,2,251,411,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,87,8,30
R_12,107,242,0,0,0,329,227,0,0,0,0,0,0,4,0,53,82,3,1,298,298,2,0,72,1550,117,3296,0,19,0,0,0,1,1037,0,0,1118,2,376,243,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,0,67,0,0,75
R_13,58,83,1,0,0,73,165,0,0,0,0,0,0,1,0,24,63,3,14,406,405,164,340,232,1980,662,3145,0,99,0,0,13,27,1763,351,4,1404,22,421,362,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,97,0,43


In [7]:
scalers = ut.load_data(home_path + 'all_process/scaling_info.pkl', 'pickle')

In [9]:
before_md_processed = pp.handle_repeated_columns(before_md[scalers['columns']])[1]
before_amc_processed = pp.handle_repeated_columns(before_amc[scalers['columns']])[1]
after_amc_processed = pp.handle_repeated_columns(after_amc[scalers['columns']])[1]

In [25]:
before_md_scaled = pd.DataFrame(scalers['power_scalers']['Meta_Datasets'].transform(before_md_processed), columns=scalers['columns'])
before_amc_scaled = pd.DataFrame(scalers['power_scalers']['AMC'].transform(before_amc_processed), columns=scalers['columns'])
after_amc_scaled = pd.DataFrame(scalers['power_scalers']['AMC'].transform(after_amc_processed), columns=scalers['columns'])

### 7월 3일
from sklearn.preprocessing import PowerTransformer

amc = pd.concat([before_amc_processed, after_amc_processed]).reset_index(drop=True)
scaler = PowerTransformer().fit(amc)
amc_scaled = pd.DataFrame(scaler.transform(amc), columns=scalers['columns'])
amc_scaled.head()

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_data.py:2982: RuntimeWarning:

divide by zero encountered in log



,ABI3BP,ACTA2,ACTC1,ACTG2,ADA,ADAM33,ADAMTS18,ADCY5,ADD2,ADH1B,ADH6,AFAP1,AGAP11,AHNAK2,AHR,ANLN,ANXA2,ANXA8,AOX1,APLP1,AREG,ART5,ASPM,ATAD2,ATAD3B,ATOH8,ATP1A2,AURKA,AURKB,B3GAT1,BAMBI,BARX2,BCL2A1,BCR,BEX1,BGN,BHLHE40,BHMT,BLK,BRSK1,...,TSC22D3,TTK,TUBB2B,TWIST1,TXLNGY,UBE2C,UBE2T,UCA1,UCHL1,UGT1A1,UHRF1,UNC119B,UPK1A,UPK2,UPK3A,VRK1,VSIG8,VSNL1,VWA5B2,WARS,WBSCR17,WDR66,WFDC8,XAF1,XBP1,XPC,YBX2,ZBED2,ZCCHC12,ZEB1,ZEB2,ZMAT3,ZNF385B,ZNF43,ZNF493,ZNF592,ZNF615,ZNF626,ZNF697,ZNF764
0,-0.816346,1.332332,-0.690270,-1.205900,-1.288922,-0.971626,-0.13484,-0.696974,-0.466206,-0.639423,-0.715694,-1.358561,-1.402005,-0.911396,-0.745227,-1.534698,0.302941,-0.231323,-0.431577,-0.774191,-1.193170,-0.31311,-1.649698,-1.616001,0.633269,-0.952955,-1.057872,-1.445692,-1.698865,-0.569129,-1.412885,-1.087989,-0.842406,-0.000308,-0.463529,0.777691,0.915052,-1.043700,-0.19245,-1.159445,...,0.951115,-1.344356,1.629934,-0.700782,-1.000368,1.151937,-1.314351,0.682164,-0.075410,-0.752608,-1.573985,-1.429644,0.566526,-0.635752,-0.200727,-1.372913,-0.494169,-0.788336,1.034838,-1.817369,-0.461052,-0.289094,-0.730175,-0.218768,1.179742,-1.535671,-0.549098,-0.517641,0.0,-1.142605,2.126338,1.774738,-0.408084,-1.651561,-1.561655,-2.025351,-1.156792,-1.212582,-1.101092,-1.094212
1,1.349533,-0.689682,-0.690270,-1.205900,0.235613,0.128142,-0.13484,1.622064,-0.466206,-0.639423,-0.715694,0.205561,0.579786,0.106939,1.031284,0.953535,-1.606139,-1.720717,2.432305,1.415224,-0.037496,-0.31311,1.233398,1.286239,0.053347,0.277976,-0.049178,1.030368,1.319396,1.834626,1.559449,0.758923,0.861733,0.351525,-0.463529,-0.181648,0.190273,1.571198,-0.19245,1.028508,...,-1.118157,0.549250,0.356807,1.714665,0.235726,0.694596,0.832149,-0.754141,-0.039428,1.615187,0.621555,0.284771,0.482090,0.864132,0.712026,0.605027,1.830909,-0.788336,1.655717,-0.116625,-0.461052,0.502247,-0.730175,0.978509,-0.353746,0.374568,1.893174,-0.517641,0.0,0.717738,1.169263,-0.195336,2.273328,0.767357,0.383080,0.002127,0.697765,-1.059746,0.387395,0.281818
2,-0.816346,-1.355276,-0.690270,-1.205900,0.369340,0.128142,-0.13484,-0.696974,-0.466206,0.908378,1.678069,-1.159838,0.289671,1.068103,0.862001,1.688638,0.131262,0.639226,-0.431577,-0.774191,0.935535,-0.31311,-1.768182,0.541841,-0.400666,-0.523120,-0.049178,-1.445692,-0.794760,-0.569129,1.020579,-1.087989,1.017848,-0.951910,-0.463529,-0.763264,0.193465,1.260604,-0.19245,-1.159445,...,1.264326,-1.344356,-0.700743,-0.700782,-1.000368,1.030581,-1.314351,1.354129,0.796874,0.055760,-1.055281,-1.429644,1.322641,1.510658,1.339905,1.027277,-0.494169,1.579332,-0.599710,-0.278072,-0.461052,-0.982520,-0.730175,-0.016179,-0.441419,-0.378323,-0.549098,-0.517641,0.0,-1.142605,-1.386479,-0.005173,-0.408084,-1.204264,0.530723,0.002127,-1.156792,-1.468237,-1.101092,0.422811
3,1.095658,-0.000280,-0.690270,0.254222,0.174161,0.128142,-0.13484,-0.696974,2.064119,1.952181,-0.715694,1.322506,-0.740495,1.414320,-0.533834,1.548130,1.446528,0.842455,-0.431577,-0.774191,1.861670,-0.31311,0.544304,1.437097,0.023287,0.730064,0.638297,1.357938,0.189667,-0.569129,-1.096025,1.094788,1.325042,0.820669,-0.463529,0.218589,1.005117,-0.164630,-0.19245,-0.107025,...,0.551998,1.023322,-1.211765,1.415464,0.893257,0.919520,1.336033,1.323148,1.220088,-0.752608,1.408811,0.054946,-0.651900,-0.531537,-0.499962,1.123678,-0.494169,-0.788336,1.796671,0.843855,-0.461052,0.502247,-0.730175,1.148660,-0.109625,-0.134595,-0.549098,1.925052,0.0,0.458793,0.542016,-0.175597,2.487554,0.584580,0.472586,0.559933,0.518280,0.423178,0.988186,0.252864
4,-0.816346,-0.317121,0.767217,0.420934,0.560370,0.830429,-0.13484,1.082664,-0.466206,-0.639423,1.634156,0.585984,0.579786,0.317822,0.341471,0.828241,0.030338,0.515264,-0.431577,0.248583,1.278865,-0.31311,0.231404,-0.064770,-0.023151,1.120899,1.029977,-0.172811,0.016377,-0.569129,0.608920,-1.087989,0.179731,-0.312377,-0.463529,0.087020,0.078263,0.481391,-0.19245,0.795143,...,0.464450,0.543032,-0.434819,1.093845,-1.000368,0.305802,0.496795,-0.146950,0.197263,1.027168,

In [13]:
model = ut.load_data(home_path + 'all_process/model_Meta_Datasets_withAMC.pkl', 'pickle')

In [26]:
before_md_prob = model['model'].predict_proba(before_md_scaled[model['columns']])[:, 1]
before_amc_prob = model['model'].predict_proba(before_amc_scaled[model['columns']])[:, 1]
after_amc_prob = model['model'].predict_proba(after_amc_scaled[model['columns']])[:, 1]

div_amc_prob = np.concatenate((before_amc_prob, after_amc_prob))

#### 7월 3일
amc_prob = model['model'].predict_proba(amc_scaled[model['columns']])[:, 1]

In [ ]:
pd.DataFrame({'prob': after_amc_prob})

In [15]:
after_amc_response = [1.0] * len(after_amc)
#total_amc_prob = np.concatenate((before_amc_prob, after_amc_prob))
total_amc_response = list(before_amc['response']) + after_amc_response

In [17]:
labels = []
#for la, lst in zip(['M', 'VTotal', 'VBefore', 'VAfter'], [list(before_md['response']), total_amc_response, list(before_amc['response']), after_amc_response]):
for la, lst in zip(['M', 'V'], [list(before_md['response']), total_amc_response]):
  tmp = np.array([f'{la}_NR'] * len(lst))
  tfs_1 = np.array(lst) == 1.0
  tmp[tfs_1] = f'{la}_R'
  labels += list(tmp) 
print(labels)

['M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR', 'M_NR',

In [19]:
#probs = np.concatenate((before_md_prob, total_amc_prob, before_amc_prob, after_amc_prob))
#fe.plot_box(probs, 'y', labels, 'Validations of 4 types of Datasets')

### 7월 3일
probs = np.concatenate((before_md_prob, amc_prob))
fe.plot_box(probs, 'y', labels, 'Validations of 2 types of Datasets')

In [20]:
important_cols = ['PCDHB9', 'HCFC1R1', 'NT5E', 'TNFAIP8', 'POU2F2']

In [ ]:
before_md_scaled.loc[np.array(before_md['response']) == 1.0][important_cols].describe()

In [ ]:
before_amc_scaled.loc[np.array(before_amc['response']) == 1.0][important_cols].describe()

In [ ]:
after_amc_scaled[important_cols].describe()

In [21]:
import plotly.figure_factory as ff

def draw_multiple_distplots(df_list, truth_listoflist, var_name, data_labels):
  print(var_name)
  data_list = []
  label_list = []
  for i, la in enumerate(data_labels):
    tmp = df_list[i][var_name].values
    for tr in [0.0, 1.0]:
      tfs = np.array(truth_listoflist[i]) == tr
      if sum(tfs) > 0:
        data_list.append(tmp[tfs])
        label_list.append(f'{la}_{tr}')
  fig = ff.create_distplot(data_list, label_list, bin_size=.2, show_hist=False)
  fig.show()

In [ ]:
df_list = [before_md_scaled, before_amc_scaled, after_amc_scaled]
truth_listoflist = [list(before_md['response']), list(before_amc['response']), [1.0] * len(after_amc)]
data_labels = ['M', 'VBefore', 'VAfter']

md_df_list = [before_md_scaled, after_amc_scaled]
md_truth_listoflist = [list(before_md['response']), [1.0] * len(after_amc)]
md_data_labels = ['M', 'VAfter']

amc_df_list = [before_amc_scaled, after_amc_scaled]
amc_truth_listoflist = [list(before_amc['response']), [1.0] * len(after_amc)]
amc_data_labels = ['VBefore', 'VAfter']

In [22]:
### 7월 3일
df_list = [before_md_scaled, amc_scaled]
truth_listoflist = [list(before_md['response']), total_amc_response]
data_labels = ['M', 'V']

In [ ]:
after_amc.shape

In [32]:
for c in important_cols:
  draw_multiple_distplots([before_md], truth_listoflist[:1], c, data_labels[:1])

PCDHB9


HCFC1R1


NT5E


TNFAIP8


POU2F2


In [35]:
for c in important_cols:
  draw_multiple_distplots([amc], truth_listoflist[1:], c, data_labels[1:])

PCDHB9


HCFC1R1


NT5E


TNFAIP8


POU2F2


In [23]:
#draw_multiple_distplots(md_df_list, md_truth_listoflist, important_cols[0], md_data_labels)
#draw_multiple_distplots(amc_df_list, amc_truth_listoflist, important_cols[0], amc_data_labels)
for c in important_cols:
  draw_multiple_distplots(df_list, truth_listoflist, c, data_labels)

PCDHB9


HCFC1R1


NT5E


TNFAIP8


POU2F2


In [27]:
pd.DataFrame({'Truth': total_amc_response, 'DivScaled_Score': div_amc_prob, 'OneScaled_Score': amc_prob})

,Truth,DivScaled_Score,OneScaled_Score
0,1.0,0.770437,0.770437
1,1.0,0.148232,0.148232
2,1.0,0.770437,0.770437
3,1.0,0.554978,0.554978
4,1.0,0.770437,0.770437
5,1.0,0.554978,0.554978
6,1.0,0.770437,0.770437
7,1.0,0.770437,0.770437
8,0.0,0.770437,0.770437
9,0.0,0.554978,0.554978


In [30]:
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score

average_precision_score(total_amc_response, div_amc_prob), roc_auc_score(total_amc_response, div_amc_prob)

(0.47362872915198495, 0.5775510204081633)

In [29]:
average_precision_score(total_amc_response, amc_prob), roc_auc_score(total_amc_response, amc_prob)

(0.4947980641531975, 0.6448979591836734)

In [31]:
div_amc_pred = np.zeros(len(div_amc_prob))
div_amc_pred[div_amc_prob >= 0.5] = 1.0
amc_pred = np.zeros(len(amc_prob))
amc_pred[amc_prob >= 0.5] = 1.0
accuracy_score(total_amc_response, div_amc_pred), accuracy_score(total_amc_response, amc_pred)

(0.5, 0.5357142857142857)

In [ ]:
after_amc[important_cols].loc[after_amc_prob < 0.5]

In [ ]:
after_amc[important_cols].loc[after_amc_prob >= 0.5]

In [ ]:
before_amc[important_cols].loc[before_amc['response'] == 1.0].describe()

In [ ]:
before_md.shape